In [348]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [349]:
df_initial = pd.read_csv('./input/listings_summary.csv')

In [350]:

# checking shape
print("The dataset has {} rows and {} columns.".format(*df_initial.shape))

# ... and duplicates
print("It contains {} duplicates.".format(df_initial.duplicated().sum()))

The dataset has 22552 rows and 96 columns.
It contains 0 duplicates.


In [351]:
# check the columns we currently have
df_initial.columns


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [352]:
# define the columns we want to keep
columns_to_keep = ['id', 'description', 'experiences_offered', 'host_has_profile_pic', 'host_response_rate', 'host_is_superhost', 'host_identity_verified', 'host_listings_count',  'neighbourhood_group_cleansed', 
                   'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',  
                   'bedrooms', 'bed_type', 'amenities', 'square_feet', 'price', 'cleaning_fee', 'space',
                   'security_deposit', 'guests_included', 'extra_people', 'minimum_nights', 'number_of_reviews', 'review_scores_rating',
                   'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',  'reviews_per_month']
df_raw = df_initial[columns_to_keep].set_index('id')
print("The dataset has {} rows and {} columns - after dropping irrelevant columns.".format(*df_raw.shape))

The dataset has 22552 rows and 31 columns - after dropping irrelevant columns.


In [353]:
#cleaning price columns 

# checking Nan's in "price" column
df_raw.price.isna().sum()

# Nan's in "cleaning_fee" column
df_raw.cleaning_fee.isna().sum()
#There are plenty of Nan's. It's more than likely that these hosts do not charge any extra cleaning fee. 
#So let's simply replace these null values with $0.00
df_raw.cleaning_fee = df_raw.cleaning_fee.fillna('$0.00')

#The same is true for the security_deposit
df_raw.security_deposit.isna().sum()
df_raw.security_deposit = df_raw.security_deposit.fillna('$0.00')

# checking Nan's in "extra_people" column
df_raw.extra_people.isna().sum()

#Let's remove the dollar signs in all four columns and convert the string values into numerical ones:
df_raw.price = df_raw.price.str.replace('$', '').str.replace(',', '').astype(float)
df_raw.cleaning_fee = df_raw.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float)
df_raw.security_deposit = df_raw.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)
df_raw.extra_people = df_raw.extra_people.str.replace('$', '').str.replace(',', '').astype(float)



In [354]:
#removing outliers from "price" column
price_statistics = df_raw.price.describe()
df_raw = df_raw.drop(df_raw[abs(df_raw.price - price_statistics["mean"]) > 3*price_statistics["std"]].index)


In [355]:
#dealing with NaN's
df_raw.isna().sum()

# drop columns with too many Nan's
df_raw.drop(columns=['square_feet', 'space', 'host_response_rate'], inplace=True)

#droping rows with reviews, bathrooms and bedrooms as NaN - I think that those are too imnportant to drop 
df_raw.dropna(subset=["review_scores_rating", "bathrooms", "bedrooms"], inplace=True)

# replace host connected columns Nan's with no or 0
df_raw.host_has_profile_pic = df_raw.host_has_profile_pic.fillna(value='f')
df_raw.host_is_superhost = df_raw.host_is_superhost.fillna(value='f')
df_raw.host_identity_verified = df_raw.host_identity_verified.fillna(value='f')
df_raw.host_listings_count = df_raw.host_listings_count.fillna(value=0)

#dropping "experiences_offered" column cause all values are "none"
df_raw.experiences_offered.unique()
df_raw.drop(columns=['experiences_offered'], inplace=True)

In [356]:
print("The dataset has {} rows and {} columns - after having dealt with missing values.".format(*df_raw.shape))

The dataset has 18107 rows and 27 columns - after having dealt with missing values.


In [357]:
#Distance to Centroid of Berlin
# adding new column "distance" distance to Berlin Centre 
from geopy.distance import great_circle
def distance_to_mid(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    accommodation = (lat, lon)
    return great_circle(berlin_centre, accommodation).km
df_raw['distance'] = df_raw.apply(lambda x: distance_to_mid(x.latitude, x.longitude), axis=1)

In [358]:
#Lodging size 

#One of the most important pieces of information for predicting the rate is the size. 
#Since the column square_feet was heavily filled with null values
#Let's check, if the column description reveals any information about size instead
df_raw.description.isna().sum()
# extract numbers 
df_raw['size'] = df_raw['description'].str.extract('(\d{2,3}\s?[smSM])', expand=True)
df_raw['size'] = df_raw['size'].str.replace("\D", "")
# change datatype of size into float
df_raw['size'] = df_raw['size'].astype(float)
print('NaNs in size_column absolute:     ', df_raw['size'].isna().sum())
print('NaNs in size_column in percentage:', round(df_raw['size'].isna().sum()/len(df_raw),3), '%')
# drop description column
df_raw.drop(['description'], axis=1, inplace=True)
#Predicting missing values with regression
# filter out sub_df to work with
sub_df = df_raw[['accommodates', 'bathrooms', 'bedrooms',  'price', 'cleaning_fee', 
                 'security_deposit', 'extra_people', 'guests_included', 'distance', 'size', 'review_scores_rating']]
# split datasets
train_data = sub_df[sub_df['size'].notnull()]
test_data  = sub_df[sub_df['size'].isnull()]

# define X
X_train = train_data.drop('size', axis=1)
X_test  = test_data.drop('size', axis=1)

# define y
y_train = train_data['size']

# import Linear Regression
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit model to training data
linreg.fit(X_train, y_train)

# making predictions
y_test = linreg.predict(X_test)
y_test = pd.DataFrame(y_test)
y_test.columns = ['size']

# make the index of X_test to an own dataframe
prelim_index = pd.DataFrame(X_test.index)
prelim_index.columns = ['prelim']

# ... and concat this dataframe with y_test
y_test = pd.concat([y_test, prelim_index], axis=1)
y_test.set_index(['prelim'], inplace=True)
new_test_data = pd.concat([X_test, y_test], axis=1)
# combine train and test data back to a new sub df
cols = train_data.columns.tolist()
cols = cols[:-2] + cols[-1:] + cols[-2:-1]
train_data = train_data[cols]
sub_df_new = pd.concat([new_test_data, train_data], axis=0)
# prepare the multiple columns before concatening
df_raw.drop(['accommodates', 'bathrooms', 'bedrooms',  'price', 'cleaning_fee', 
                 'security_deposit', 'extra_people', 'guests_included', 'distance', 'size', 'review_scores_rating'], 
            axis=1, inplace=True)
# concate back to complete dataframe
df = pd.concat([sub_df_new, df_raw], axis=1)
#To be on the safe side, let’s remove all outliers
size_statistics = df["size"].describe()
df = df.drop(df[abs(df["size"]- size_statistics["mean"]) > 3*size_statistics["std"]].index)

NaNs in size_column absolute:      9153
NaNs in size_column in percentage: 0.505 %


In [370]:
#Split berlin into sectors 
east_berlin = ['Mitte', 'Pankow', 'Friedrichshain-Kreuzberg', 'Treptow - Köpenick', 'Lichtenberg', 'Marzahn - Hellersdorf']
west_berlin = ['Tempelhof - Schöneberg', 'Charlottenburg-Wilm.', 'Neukölln', 'Steglitz - Zehlendorf', 'Reinickendorf', 'Spandau']
df["is_in_east_berlin"] = df.neighbourhood_group_cleansed.isin(east_berlin)


In [375]:
#Replace strings with booleans
df.replace('t', True, inplace=True)
df.replace('f', False, inplace=True)


In [376]:
df.dtypes

accommodates                      int64
bathrooms                       float64
bedrooms                        float64
price                           float64
cleaning_fee                    float64
security_deposit                float64
extra_people                    float64
guests_included                   int64
distance                        float64
review_scores_rating            float64
size                            float64
host_has_profile_pic               bool
host_is_superhost                  bool
host_identity_verified             bool
host_listings_count             float64
neighbourhood_group_cleansed     object
latitude                        float64
longitude                       float64
property_type                    object
room_type                        object
bed_type                         object
amenities                        object
minimum_nights                    int64
number_of_reviews                 int64
instant_bookable                   bool
